In [1]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

import re, string
import pandas as pd
import numpy as np
import csv
import time

import jieba

import string
from collections import namedtuple

In [2]:
# defining a function to remove the noises
# we don't want to take comma into account
# because each title is just a sentence, we just delete some very very common word
def noise_remove(text):

    # remove punctuations
    punctuation = r',，“”‘;:.。 "\''
    
    # remove the one letter character
    stripped_text=re.sub('的','',text)
    
    # remove the one letter character
    stripped_text=re.sub('了','',stripped_text)
    
    # remove the one letter character
    stripped_text=re.sub('得','',stripped_text)
    
    # remove the one letter character
    stripped_text=re.sub('地','',stripped_text)
    
    # remove the one letter character
    stripped_text=re.sub('在','',stripped_text)
    
    # remove punctuations
    stripped_text.translate(str.maketrans('', '', string.punctuation))
    stripped_text = re.sub(r'[{}]+'.format(punctuation),'',stripped_text)
    
    # remove the white space and make it lower case
    stripped_text = stripped_text.strip()
    
    return stripped_text

In [3]:
# defining a function to do the sentiment analysis
def sentiment_analysis(words, sentimentlist):
    
    # count the sentimental words
    senti=0
    
    for word in words:
        # if the word is in sentimental list, add one value
        if word in sentimentlist:
            senti = senti + 1            
            
    return senti

In [4]:
# defining a function to do the punctuations analysis
def punctuation_analysis(words, punctuationlist):
    
    # count the punctuations
    punc=0

    for word in words:
        # if the word is in punctuation list, add one value
        if word in punctuationlist:
            punc = punc + 1            
            
    return punc

In [5]:
# get the number of digits in the title
def number_analysis(words):
    
    numbers = [v for v in words if str(v).isdigit()]
    
    return numbers

In [6]:
# get the digit and the word after the digit
def number_advanced_analysis(words):
    
    phrase = []
    for word in words:
        if str(word).isdigit():
            if (segs.index(word)+1 < len(words)):
                phrase.append(segs[segs.index(word)]+segs[segs.index(word)+1])
    
    if (len(phrase)==0):
        return None
    else:
        return phrase

In [7]:
# determining whether it is a question or not
def containing_questions(words, questionlist):
    
    count = 0
    for word in words:
        # if the word is in question list, add the positive evaluation
        if word in questionlist:
            count = count + 1
    if count > 0:
        return 1
    else:
        return 0

In [8]:
# count the number of characters in a title
def str_count(s):

    count_en = count_dg = count_sp = count_zh = count_pu = 0
    s_len = len(s)
    return(s_len)
#     for c in s:
#         if c in string.ascii_letters:
#             count_en += 1
#         elif c.isdigit():
#             count_dg += 1
#         elif c.isspace():
#             count_sp += 1
#         elif c.isalpha():
#             count_zh += 1
#         else:
#             count_pu += 1
#     total_chars = count_zh + count_en + count_sp + count_dg + count_pu
#     if total_chars == s_len:
#         return namedtuple('Count', ['total', 'zh', 'en', 'space', 'digit', 'punc'])(s_len, count_zh, count_en, count_sp, count_dg, count_pu)
# #         return total_chars
#     else:
#         print('Something is wrong!')
#         return None
#     return None

In [9]:
# reading the excel file
xls = pd.read_excel('../data/chinese_senti.xlsx')
senti_list = list(xls['senti'])
# print(senti_list)


# reading the excel file
xls = pd.read_excel('../data/punctuations.xlsx')
punc_list = list(xls['punc'])
# print(punc_list)

# reading the titles
# titles = pd.read_excel('final_bilibili_perfect_food.xlsx')
# titles = pd.read_excel('final_bilibili_perfect_fun.xlsx')
# titles = pd.read_excel('final_bilibili_perfect_fitness.xlsx')
titles = pd.read_excel('../data/final_bilibili_perfect_makeup.xlsx')

# reading the excel file
ques = pd.read_excel('../data/question_marks.xlsx')
ques_list = list(ques['question_mark'])


In [96]:
tits = list(titles['title'])
views = list(titles['view'])
follows = list(titles['follower'])
likes = list(titles['like'])
collects = list(titles['collect'])

# print(tits[0])
content = noise_remove(tits[0])
# print(content)

col= [['title','view','follows','likes','collects','senti_result','punc_result','num_of_numbers','is_question','total_char','num_phrase']]
i = 0

for tit in tits:

#     print(tit)
    
    content = noise_remove(tit)
    segs = jieba.cut(content, cut_all=False)
    segs = list(segs)

    one_row = []
    
    one_row.append(tit)
    
    one_row.append(views[i])
    
    one_row.append(follows[i])
    
    one_row.append(likes[i])
    
    one_row.append(collects[i])
    
    # calling the sentiment analysis function
    senti_result = sentiment_analysis(segs, senti_list)
    one_row.append(senti_result)

    # calling the punctuation function
    punc_result = punctuation_analysis(segs, punc_list)
    one_row.append(punc_result)

    # knowing the number of digits
    num_result = number_analysis(segs)
    num_numbers = len(num_result)
    one_row.append(num_numbers)

    # determining whether it is a question
    question_result = containing_questions(segs,ques_list)
    one_row.append(question_result)  
    
    # counting the lenth of title
    string_info = str_count(tit)
    one_row.append(string_info)
    
    # getting the number 
    num_phrase = number_advanced_analysis(segs)
    one_row.append(num_phrase)

    col.append(one_row)
    i=i+1
    


In [97]:
# concert list to dataframe
col = pd.DataFrame(col)

# write the csv file
# col.to_csv('../data/results_food.csv', header=False)
# col.to_csv('../data/results_fun.csv', header=False)
# col.to_csv('../data/results_fitness.csv', header=False)
col.to_csv('../data/results_makeup.csv', header=False)